This notebook will be used for the capstone project

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import requests
import matplotlib.pyplot as plt
import matplotlib.cm as cm 
import matplotlib.colors as colors
import pgeocode
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup

Make a request to get the content of the website


In [2]:
webpage_response = requests.get("http://en.turkcewiki.org/wiki/List_of_postal_codes_of_Canada:_M")
webpage = webpage_response.content
#Create a BeautifulSoup object
soup =BeautifulSoup(webpage,"html.parser")

Create a empty list and find the table data within the webpage 

In [3]:
table_contents = []
#Find the table data
table = soup.find('table')

Iterate through the table data, here create a dictionary called cell having three keys: PostalCode, Borough and Neighborhood.

The postcode value will be extrated from row.p.text. The borough and Neighborhood will be extracted from row.span.txt
Cell will be conformed only where the borough and the neigborhoods are assigned. 

In the table, the neighborhood are inside parentheses and if the are more than one neighborhhood for one postal code they are separated by a /. To obtain the neighborhoods without the parentheses and separated by a comma, use the split and replace methods.

In [4]:
for row in table.findAll('td'):
    #Create a dictionary with 3 keys: PostalCode, Borough and Neighborhood
    cell = {}
    #Not including "Not assigned" values
    if row.span.text=='Not assigned':
        pass
    else:
        #Get the 3 PostalCode Letters
        cell['PostalCode'] = row.p.text[:3]
        #Get the Borough
        cell['Borough'] = (row.span.text).split('(')[0]
        #Get the neighborhoods, separate them with a ","
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

Created the dataframe and clean the data 

In [5]:
#Create the dataframe        
df=pd.DataFrame(table_contents)
#Rename the Borough values
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
print(df.head())
df.shape

  PostalCode           Borough                      Neighborhood
0        M3A        North York                         Parkwoods
1        M4A        North York                  Victoria Village
2        M5A  Downtown Toronto         Regent Park, Harbourfront
3        M6A        North York  Lawrence Manor, Lawrence Heights
4        M7A      Queen's Park     Ontario Provincial Government


(103, 3)

Obtain the latitudes and longitudes for every PostalCode using the pgeocode module

In [6]:
geolocator = pgeocode.Nominatim("CA")
latitudes = []
longitudes = []
for code in df["PostalCode"]    :          
    location=geolocator.query_postal_code(code) 
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)
df["Latitude"] = latitudes
df["Longitude"] = longitudes
df.head()



,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889
